# **Random Forest**
- Using data that was not annotated
- Training it on the unbalanced data
- Using avgpool
- 4 second window

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
# Standard libraries
import numpy as np
import pandas as pd
import time
import os

# For audio
from IPython.display import Audio
import librosa

# For preprocessing
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import OneHotEncoder
import tensorflow as tf

# For modeling
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report

# Operational
from tqdm import tqdm
import matplotlib.pyplot as plt
import seaborn as sns
import pickle
import scipy.ndimage
import pygame
import time
from scipy.signal import butter, filtfilt
import random

pygame 2.6.1 (SDL 2.28.4, Python 3.10.12)
Hello from the pygame community. https://www.pygame.org/contribute.html


In [3]:
pkl_path = '/content/drive/My Drive/Final-Year-Project/Dataset/Project-V4/feature-extraction/NotAnnotated/Regular/NotAveragePooled/split_features_3s_all_2D.pkl'

# Load the pickle file
with open(pkl_path, 'rb') as file:
    data = pickle.load(file)

In [4]:
train_data = data['train'].copy()
val_data = data['val'].copy()

In [5]:
train_data

{'melspectrogram': array([[[2.70414866e-03, 2.79165306e-04, 2.78151976e-06, ...,
          1.06072697e-14, 6.16832790e-08, 5.78963081e-06],
         [2.85087147e-03, 2.95306839e-04, 3.42857310e-06, ...,
          1.12205350e-14, 6.15759939e-08, 5.77909589e-06],
         [3.04498922e-03, 3.15533762e-04, 4.71718391e-06, ...,
          1.25414211e-14, 6.13728398e-08, 5.75930686e-06],
         ...,
         [4.71950382e-06, 6.96979924e-07, 9.83989841e-06, ...,
          1.16200449e-08, 1.17388327e-08, 3.10218327e-07],
         [4.43196142e-06, 4.70733005e-07, 1.06044568e-07, ...,
          1.11483545e-10, 1.11642392e-09, 9.85757182e-08],
         [4.34125244e-06, 4.54310514e-07, 3.83112779e-09, ...,
          1.07090598e-13, 3.56324655e-10, 3.34282947e-08]],
 
        [[1.22058890e-04, 9.34677081e-06, 3.22876749e-15, ...,
          9.62427203e-15, 2.13093339e-07, 5.59486302e-06],
         [1.24266710e-04, 9.51594868e-06, 1.18513276e-14, ...,
          1.10033380e-14, 2.14718425e-07, 5.6383

In [6]:
train_labels = train_data['label'].copy()
temp = train_data.copy()
del temp['label']
tr_features = temp

In [7]:
val_labels = val_data['label'].copy()
temp = val_data.copy()
del temp['label']
v_features = temp

# Shuffle Data

In [8]:
def shuffle_data(input_label, input_features):
  input_len = len(input_label)
  np.random.seed(1826)
  input_indices = np.random.permutation(input_len)
  input_features = {key: np.array([input_features[key][i] for i in input_indices]) for key in input_features} # dictionary comprehension
  input_label = np.array([input_label[i] for i in input_indices])

  return input_label, input_features

In [9]:
train_y, train_features = shuffle_data(train_labels, tr_features)

In [10]:
val_y, val_features = shuffle_data(val_labels, v_features)

In [11]:
display(train_y.shape)
display(train_y[:15])

(10981,)

array([ 0,  2, 12, 19,  1, 16, 12, 12,  2, 10,  1, 17, 17, 16,  5],
      dtype=int32)

In [12]:
for key in train_features.keys():
  display(key)
  display(train_features[key].shape)
  display(train_features[key][0])

'melspectrogram'

(10981, 60, 173)

array([[2.86515150e-03, 6.80767231e-04, 2.97909150e-11, ...,
        1.20800270e-12, 1.54432492e-05, 4.30832110e-04],
       [2.90407872e-03, 6.90050163e-04, 6.33079126e-11, ...,
        9.03021194e-12, 1.60103375e-05, 4.46285096e-04],
       [2.97047890e-03, 7.05829968e-04, 3.58781896e-09, ...,
        4.05444638e-10, 1.69596877e-05, 4.72895689e-04],
       ...,
       [3.84271783e-06, 2.62903578e-06, 3.10762521e-06, ...,
        2.43702737e-06, 9.06983186e-06, 1.78433534e-04],
       [7.97103049e-07, 2.02830843e-07, 2.81801248e-08, ...,
        1.21953893e-08, 5.36466534e-06, 1.55784888e-04],
       [3.61111170e-07, 8.48842786e-08, 2.33122790e-11, ...,
        9.66402053e-12, 5.04421756e-06, 1.46645771e-04]])

'mfcc'

(10981, 20, 173)

...

(10981, 12, 173)

array([[0.49322796, 0.55023208, 0.99232902, ..., 0.72210309, 0.83299871,
        0.63234777],
       [1.        , 1.        , 1.        , ..., 0.94905946, 0.97092678,
        0.57756803],
       [0.71974691, 0.88993172, 0.96673663, ..., 1.        , 1.        ,
        0.87034696],
       ...,
       [0.37971051, 0.70089198, 0.86866288, ..., 0.73178094, 0.69199241,
        0.96665332],
       [0.31718219, 0.57008808, 0.98001273, ..., 0.5537603 , 0.65570829,
        0.87120081],
       [0.32763275, 0.44132466, 0.99494184, ..., 0.5402766 , 0.98134254,
        1.        ]])

'rms'

(10981, 1, 173)

array([[0.00039074, 0.00042852, 0.0004804 , 0.00054854, 0.00058784,
        0.00059393, 0.00059617, 0.00046273, 0.00041349, 0.00041963,
        0.00041193, 0.00041588, 0.00042323, 0.000566  , 0.00061378,
        0.00074525, 0.00152645, 0.00249982, 0.00342579, 0.01243992,
        0.01289345, 0.01355664, 0.01639647, 0.01172002, 0.01117699,
        0.01018599, 0.00364752, 0.0012906 , 0.00091503, 0.00064468,
        0.00050603, 0.0004332 , 0.00041633, 0.00040445, 0.00039667,
        0.00039603, 0.00039678, 0.00040109, 0.00052812, 0.00073974,
        0.00167491, 0.00551012, 0.01040052, 0.02760901, 0.02811749,
        0.02768876, 0.02625788, 0.00595194, 0.00222059, 0.00113607,
        0.00076726, 0.00063663, 0.00062657, 0.00285686, 0.01670538,
        0.01927643, 0.02217778, 0.02311721, 0.01691305, 0.01483164,
        0.01245705, 0.01647372, 0.02695421, 0.03781732, 0.04618729,
        0.04510737, 0.03964701, 0.02910161, 0.00945323, 0.00467422,
        0.00572438, 0.02610471, 0.03316663, 0.03

In [13]:
display(val_y.shape)
display(val_y[:15])

(2928,)

array([ 1,  0,  3, 13, 15,  7,  0, 11, 17, 18, 10,  0,  1,  2,  2],
      dtype=int32)

In [14]:
for key in val_features.keys():
  display(key)
  display(val_features[key].shape)
  display(val_features[key][0])

'melspectrogram'

(2928, 60, 173)

array([[3.32319924e-06, 3.85627327e-07, 1.62118227e-13, ...,
        8.35242172e-13, 1.47860481e-04, 2.38549490e-03],
       [3.39569346e-06, 3.94667132e-07, 2.45596577e-13, ...,
        1.36000466e-12, 1.49738668e-04, 2.41649617e-03],
       [3.52569295e-06, 4.10863971e-07, 8.27563479e-12, ...,
        1.52835391e-11, 1.53004160e-04, 2.47022787e-03],
       ...,
       [4.70026294e-07, 7.63783576e-08, 2.50398643e-08, ...,
        1.18084688e-07, 8.87401461e-06, 1.05132121e-04],
       [3.93884604e-07, 4.36421482e-08, 1.52936053e-10, ...,
        5.23465998e-10, 5.83651537e-06, 9.45892576e-05],
       [3.64246105e-07, 4.02502116e-08, 1.24587951e-13, ...,
        1.86082433e-13, 5.47833371e-06, 8.88705570e-05]])

'mfcc'

(2928, 20, 173)

...

(2928, 12, 173)

array([[0.02405424, 0.03210343, 0.07836496, ..., 0.01892673, 0.25222963,
        0.35473842],
       [0.10054196, 0.21368316, 0.40564924, ..., 0.02409197, 0.34999723,
        0.50214092],
       [0.1580866 , 0.14295156, 0.09562304, ..., 0.01910207, 0.36193931,
        0.58541331],
       ...,
       [0.03860641, 0.02437317, 0.02632891, ..., 0.24834281, 0.73819415,
        0.66448111],
       [0.01557225, 0.01391101, 0.03290793, ..., 0.12710531, 0.6942014 ,
        0.57520359],
       [0.02324896, 0.02448379, 0.04552641, ..., 0.02347876, 0.39079534,
        0.40234969]])

'rms'

(2928, 1, 173)

array([[0.02225015, 0.02555211, 0.0265466 , 0.02252608, 0.01568867,
        0.02629739, 0.02896822, 0.0435867 , 0.04932851, 0.04663459,
        0.04794999, 0.03810471, 0.05818411, 0.07532492, 0.07372992,
        0.07230121, 0.05424416, 0.0222149 , 0.02307186, 0.02486459,
        0.02186136, 0.03875333, 0.05505793, 0.06267009, 0.06324769,
        0.05314591, 0.03477944, 0.01649618, 0.01746471, 0.01886477,
        0.02194129, 0.023644  , 0.02064178, 0.01911475, 0.02065624,
        0.02115975, 0.02198245, 0.02571904, 0.02157723, 0.01782953,
        0.02261083, 0.02795133, 0.03256924, 0.04175666, 0.03936617,
        0.03579794, 0.04168303, 0.03811065, 0.04565651, 0.04389921,
        0.03450453, 0.03074298, 0.0218442 , 0.03266475, 0.03377284,
        0.03713641, 0.03521169, 0.0367388 , 0.03605549, 0.03023931,
        0.0293515 , 0.01836588, 0.0236658 , 0.02847057, 0.02899235,
        0.02746851, 0.02799471, 0.02754326, 0.02959297, 0.02768993,
        0.02191872, 0.02211438, 0.0183103 , 0.03

## **Random Forest Model**

In [15]:
train_results = {}
val_results = {}

### **With all the features**

In [16]:
training_features_3D = np.concatenate((train_features['mfcc'], train_features['chroma'], train_features['rms'], train_features['melspectrogram']), axis=1)
training_features = training_features_3D.reshape(training_features_3D.shape[0], -1)
training_features.shape

(10981, 16089)

In [17]:
validation_features_3D = np.concatenate((val_features['mfcc'], val_features['chroma'], val_features['rms'], val_features['melspectrogram']), axis=1)
validation_features = validation_features_3D.reshape(validation_features_3D.shape[0], -1)
validation_features.shape

(2928, 16089)

Fit the model with training data

In [18]:
rf = RandomForestClassifier(n_estimators=50, criterion='entropy')
rf.fit(training_features, train_y)

RandomForestClassifier(criterion='entropy', n_estimators=50)

In [19]:
print('Training accuracy:', rf.score(training_features, train_y))
train_results['all_features'] = rf.score(training_features, train_y)

print('Validation accuracy:', rf.score(validation_features, val_y))
val_results['all_features'] = rf.score(validation_features, val_y)

Training accuracy: 1.0
Validation accuracy: 0.3668032786885246


In [20]:
train_yhat_result = rf.predict(training_features)

print('Training classification Report \n')
print(classification_report(train_y, train_yhat_result))

Training classification Report 

              precision    recall  f1-score   support

           0       1.00      1.00      1.00      1542
           1       1.00      1.00      1.00      1114
           2       1.00      1.00      1.00       763
           3       1.00      1.00      1.00       708
           4       1.00      1.00      1.00       105
           5       1.00      1.00      1.00       177
           6       1.00      1.00      1.00       199
           7       1.00      1.00      1.00       630
           8       1.00      1.00      1.00       175
           9       1.00      1.00      1.00       314
          10       1.00      1.00      1.00       437
          11       1.00      1.00      1.00       168
          12       1.00      1.00      1.00      1098
          13       1.00      1.00      1.00       291
          14       1.00      1.00      1.00       424
          15       1.00      1.00      1.00       629
          16       1.00      1.00      1.00     

In [21]:
val_yhat_result = rf.predict(validation_features)

print('Validation classification Report \n')
print(classification_report(val_y, val_yhat_result))

Validation classification Report 

              precision    recall  f1-score   support

           0       0.32      0.74      0.45       412
           1       0.22      0.43      0.29       184
           2       0.86      0.31      0.46       717
           3       0.46      0.76      0.58       147
           4       0.00      0.00      0.00        74
           5       0.17      0.13      0.15        30
           6       0.00      0.00      0.00        29
           7       0.52      0.47      0.50       172
           8       0.17      0.02      0.03        59
           9       0.77      0.58      0.66       100
          10       0.25      0.23      0.24       117
          11       0.00      0.00      0.00        59
          12       0.34      0.60      0.43       139
          13       0.00      0.00      0.00        51
          14       0.04      0.10      0.06        30
          15       0.27      0.14      0.18       211
          16       0.25      0.19      0.22   

### **With MFCCs only**

In [22]:
training_features_3D = train_features['mfcc']
training_features = training_features_3D.reshape(training_features_3D.shape[0], -1)
training_features.shape

(10981, 3460)

In [23]:
validation_features_3D = val_features['mfcc']
validation_features = validation_features_3D.reshape(validation_features_3D.shape[0], -1)
validation_features.shape

(2928, 3460)

In [24]:
rf = RandomForestClassifier(n_estimators=50, criterion='entropy')
rf.fit(training_features, train_y)

RandomForestClassifier(criterion='entropy', n_estimators=50)

In [25]:
print('Training accuracy:', rf.score(training_features, train_y))
train_results['mfcc'] = rf.score(training_features, train_y)

print('Validation accuracy:', rf.score(validation_features, val_y))
val_results['mfcc'] = rf.score(validation_features, val_y)

Training accuracy: 1.0
Validation accuracy: 0.34665300546448086


### **With Mel-Spectrogram Only**

In [26]:
training_features_3D = train_features['melspectrogram']
training_features = training_features_3D.reshape(training_features_3D.shape[0], -1)
training_features.shape

(10981, 10380)

In [27]:
validation_features_3D = val_features['melspectrogram']
validation_features = validation_features_3D.reshape(validation_features_3D.shape[0], -1)
validation_features.shape

(2928, 10380)

In [28]:
rf = RandomForestClassifier(n_estimators=50, criterion='entropy')
rf.fit(training_features, train_y)

RandomForestClassifier(criterion='entropy', n_estimators=50)

In [29]:
print('Training accuracy:', rf.score(training_features, train_y))
train_results['melspectrogram'] = rf.score(training_features, train_y)

print('Validation accuracy:', rf.score(validation_features, val_y))
val_results['melspectrogram'] = rf.score(validation_features, val_y)

Training accuracy: 1.0
Validation accuracy: 0.32786885245901637


## **With Chroma Only**

In [30]:
training_features_3D = train_features['chroma']
training_features = training_features_3D.reshape(training_features_3D.shape[0], -1)
training_features.shape

(10981, 2076)

In [31]:
validation_features_3D = val_features['chroma']
validation_features = validation_features_3D.reshape(validation_features_3D.shape[0], -1)
validation_features.shape

(2928, 2076)

In [32]:
rf = RandomForestClassifier(n_estimators=50, criterion='entropy')
rf.fit(training_features, train_y)

RandomForestClassifier(criterion='entropy', n_estimators=50)

In [33]:
print('Training accuracy:', rf.score(training_features, train_y))
train_results['chroma'] = rf.score(training_features, train_y)

print('Validation accuracy:', rf.score(validation_features, val_y))
val_results['chroma'] = rf.score(validation_features, val_y)

Training accuracy: 1.0
Validation accuracy: 0.21448087431693988


## **With MFCCs and Mel-Spectrogram**

In [34]:
training_features_3D = np.concatenate((train_features['mfcc'], train_features['melspectrogram']), axis=1)
training_features = training_features_3D.reshape(training_features_3D.shape[0], -1)
training_features.shape

(10981, 13840)

In [35]:
validation_features_3D = np.concatenate((val_features['mfcc'], val_features['melspectrogram']), axis=1)
validation_features = validation_features_3D.reshape(validation_features_3D.shape[0], -1)
validation_features.shape

(2928, 13840)

In [36]:
rf = RandomForestClassifier(n_estimators=50, criterion='entropy')
rf.fit(training_features, train_y)

RandomForestClassifier(criterion='entropy', n_estimators=50)

In [37]:
print('Training accuracy:', rf.score(training_features, train_y))
train_results['mfcc_melspectrogram'] = rf.score(training_features, train_y)

print('Validation accuracy:', rf.score(validation_features, val_y))
val_results['mfcc_melspectrogram'] = rf.score(validation_features, val_y)

Training accuracy: 1.0
Validation accuracy: 0.3671448087431694


## **With MFCCs and Chroma**

In [38]:
training_features_3D = np.concatenate((train_features['mfcc'], train_features['chroma']), axis=1)
training_features = training_features_3D.reshape(training_features_3D.shape[0], -1)
training_features.shape

(10981, 5536)

In [39]:
validation_features_3D = np.concatenate((val_features['mfcc'], val_features['chroma']), axis=1)
validation_features = validation_features_3D.reshape(validation_features_3D.shape[0], -1)
validation_features.shape

(2928, 5536)

In [40]:
rf = RandomForestClassifier(n_estimators=50, criterion='entropy')
rf.fit(training_features, train_y)

RandomForestClassifier(criterion='entropy', n_estimators=50)

In [41]:
print('Training accuracy:', rf.score(training_features, train_y))
train_results['mfcc_chroma'] = rf.score(training_features, train_y)

print('Validation accuracy:', rf.score(validation_features, val_y))
val_results['mfcc_chroma'] = rf.score(validation_features, val_y)

Training accuracy: 1.0
Validation accuracy: 0.3603142076502732


## **With MFCCs and RMS**

In [42]:
training_features_3D = np.concatenate((train_features['mfcc'], train_features['rms']), axis=1)
training_features = training_features_3D.reshape(training_features_3D.shape[0], -1)
training_features.shape

(10981, 3633)

In [43]:
validation_features_3D = np.concatenate((val_features['mfcc'], val_features['rms']), axis=1)
validation_features = validation_features_3D.reshape(validation_features_3D.shape[0], -1)
validation_features.shape

(2928, 3633)

In [44]:
rf = RandomForestClassifier(n_estimators=50, criterion='entropy')
rf.fit(training_features, train_y)

RandomForestClassifier(criterion='entropy', n_estimators=50)

In [45]:
print('Training accuracy:', rf.score(training_features, train_y))
train_results['mfcc_rms'] = rf.score(training_features, train_y)

print('Validation accuracy:', rf.score(validation_features, val_y))
val_results['mfcc_rms'] = rf.score(validation_features, val_y)

Training accuracy: 1.0
Validation accuracy: 0.3650956284153005


## **With Chroma and RMS**

In [46]:
training_features_3D = np.concatenate((train_features['chroma'], train_features['rms']), axis=1)
training_features = training_features_3D.reshape(training_features_3D.shape[0], -1)
training_features.shape

(10981, 2249)

In [47]:
validation_features_3D = np.concatenate((val_features['chroma'], val_features['rms']), axis=1)
validation_features = validation_features_3D.reshape(validation_features_3D.shape[0], -1)
validation_features.shape

(2928, 2249)

In [48]:
rf = RandomForestClassifier(n_estimators=50, criterion='entropy')
rf.fit(training_features, train_y)

RandomForestClassifier(criterion='entropy', n_estimators=50)

In [49]:
print('Training accuracy:', rf.score(training_features, train_y))
train_results['chroma_rms'] = rf.score(training_features, train_y)

print('Validation accuracy:', rf.score(validation_features, val_y))
val_results['chroma_rms'] = rf.score(validation_features, val_y)

Training accuracy: 1.0
Validation accuracy: 0.2926912568306011


## **With Mel-Spectrogram and Chroma**

In [50]:
training_features_3D = np.concatenate((train_features['melspectrogram'], train_features['chroma']), axis=1)
training_features = training_features_3D.reshape(training_features_3D.shape[0], -1)
training_features.shape

(10981, 12456)

In [51]:
validation_features_3D = np.concatenate((val_features['melspectrogram'], val_features['chroma']), axis=1)
validation_features = validation_features_3D.reshape(validation_features_3D.shape[0], -1)
validation_features.shape

(2928, 12456)

In [52]:
rf = RandomForestClassifier(n_estimators=50, criterion='entropy')
rf.fit(training_features, train_y)

RandomForestClassifier(criterion='entropy', n_estimators=50)

In [53]:
print('Training accuracy:', rf.score(training_features, train_y))
train_results['melspectrogram_chroma'] = rf.score(training_features, train_y)

print('Validation accuracy:', rf.score(validation_features, val_y))
val_results['melspectrogram_chroma'] = rf.score(validation_features, val_y)

Training accuracy: 1.0
Validation accuracy: 0.3056693989071038


## **With Mel-Spectrogram and RMS**

In [54]:
training_features_3D = np.concatenate((train_features['melspectrogram'], train_features['rms']), axis=1)
training_features = training_features_3D.reshape(training_features_3D.shape[0], -1)
training_features.shape

(10981, 10553)

In [55]:
validation_features_3D = np.concatenate((val_features['melspectrogram'], val_features['rms']), axis=1)
validation_features = validation_features_3D.reshape(validation_features_3D.shape[0], -1)
validation_features.shape

(2928, 10553)

In [56]:
rf = RandomForestClassifier(n_estimators=50, criterion='entropy')
rf.fit(training_features, train_y)

RandomForestClassifier(criterion='entropy', n_estimators=50)

In [57]:
print('Training accuracy:', rf.score(training_features, train_y))
train_results['melspectrogram_rms'] = rf.score(training_features, train_y)

print('Validation accuracy:', rf.score(validation_features, val_y))
val_results['melspectrogram_rms'] = rf.score(validation_features, val_y)

Training accuracy: 1.0
Validation accuracy: 0.3541666666666667


## **With Mel-Spectrogram, MFCCs, RMS**

In [58]:
training_features_3D = np.concatenate((train_features['melspectrogram'], train_features['rms'], train_features['mfcc']), axis=1)
training_features = training_features_3D.reshape(training_features_3D.shape[0], -1)
training_features.shape

(10981, 14013)

In [59]:
validation_features_3D = np.concatenate((val_features['melspectrogram'], val_features['rms'], val_features['mfcc']), axis=1)
validation_features = validation_features_3D.reshape(validation_features_3D.shape[0], -1)
validation_features.shape

(2928, 14013)

In [60]:
rf = RandomForestClassifier(n_estimators=50, criterion='entropy')
rf.fit(training_features, train_y)

RandomForestClassifier(criterion='entropy', n_estimators=50)

In [61]:
print('Training accuracy:', rf.score(training_features, train_y))
train_results['melspectrogram_rms_mfcc'] = rf.score(training_features, train_y)

print('Validation accuracy:', rf.score(validation_features, val_y))
val_results['melspectrogram_rms_mfcc'] = rf.score(validation_features, val_y)

Training accuracy: 1.0
Validation accuracy: 0.37260928961748635


## **With Mel-Spectrogram, MFCCs, Chroma**

In [62]:
training_features_3D = np.concatenate((train_features['melspectrogram'], train_features['chroma'], train_features['mfcc']), axis=1)
training_features = training_features_3D.reshape(training_features_3D.shape[0], -1)
training_features.shape

(10981, 15916)

In [63]:
validation_features_3D = np.concatenate((val_features['melspectrogram'], val_features['chroma'], val_features['mfcc']), axis=1)
validation_features = validation_features_3D.reshape(validation_features_3D.shape[0], -1)
validation_features.shape

(2928, 15916)

In [64]:
rf = RandomForestClassifier(n_estimators=50, criterion='entropy')
rf.fit(training_features, train_y)

RandomForestClassifier(criterion='entropy', n_estimators=50)

In [65]:
print('Training accuracy:', rf.score(training_features, train_y))
train_results['melspectrogram_chroma_mfcc'] = rf.score(training_features, train_y)

print('Validation accuracy:', rf.score(validation_features, val_y))
val_results['melspectrogram_chroma_mfcc'] = rf.score(validation_features, val_y)

Training accuracy: 1.0
Validation accuracy: 0.375


# Review results from all models

In [66]:
train_results_df = pd.DataFrame(list(train_results.items()), columns=['Features', 'Train_Accuracy']).round(2)
val_results_df = pd.DataFrame(list(val_results.items()), columns=['Features', 'Val_Accuracy']).round(2)

result_df = train_results_df.merge(val_results_df, on='Features')
result_df = result_df.sort_values('Features')
result_df

,Features,Train_Accuracy,Val_Accuracy
0,all_features,1.0,0.37
3,chroma,1.0,0.21
7,chroma_rms,1.0,0.29
2,melspectrogram,1.0,0.33
8,melspectrogram_chroma,1.0,0.31
11,melspectrogram_chroma_mfcc,1.0,0.38
9,melspectrogram_rms,1.0,0.35
10,melspectrogram_rms_mfcc,1.0,0.37
1,mfcc,1.0,0.35
5,mfcc_chroma,1.0,0.36
